In [1]:
"""
The goals/steps of this project are the following:
. Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
. Apply a distortion correction to raw images.
. Use color transforms, gradients, etc., to create a threshold binary image.
. Detect lane pixels and fit to find the lane boundary
. Determine the curvature of the lane and vehicle position with respect to center.
. Warp the detected lane boundaries back onto the original image.
. Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.
"""
import cv2
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
import glob
import pickle
from ipywidgets import widgets
from IPython.display import display 
from IPython.display import Image
from ipywidgets import interactive, interact, fixed
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline
%matplotlib qt




# Camera Calibration


In [ ]:
"""Calibrating camera
Steps involved in camera calibration:
i. Arrays to store object pointsand image points from all the images
ii. convert to grayscale
iii. Find chess board corners
iv. If corners are found add image points and object points
"""
def camera_calibration(images, nx, ny):
    # Arrays to store object points and image points
    objpoints = [] # 3D points in real world space
    imgpoints = [] # 2D points i image plane
    # Prepare object points by creating 9X6 points in  array each with 3 columns for the x,y,z coordinates of each corner
    objp = np.zeros((ny*nx,3), np.float32)
    # Use numpy mgrid function to generate the coordinates 
    objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)
    
    for image in images:
        img = mpimg.imread(image)
        
        # convert image to grayscale
        gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        
        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)
        #print(ret)
        # If corners are found, add image points and object points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
            # draw and display the corners
            # img = cv2.drawChessboardCorners(img, (nx,ny),corners, ret)
            # plt.imshow(img)
            
    return cv2.calibrateCamera(objpoints, imgpoints, img.shape[0:2], None, None)
    


In [ ]:
nx = 9
ny = 6
images = glob.glob('camera_cal/calibration*.jpg')
#Checking file directory
#img = mpimg.imread('camera_cal/calibration1.jpg')
#plt.figure()
#plt.imshow(img)
#calibrate the camera
ret, mtx,dist, rvecs, tvecs = camera_calibration(images,nx,ny)


In [ ]:
global mtx,dist,nx,ny

# Correcting the distortion


In [ ]:
def camera_undistort(img, mtx, dist):
    return cv2.undistort(img, mtx, dist, None, mtx)

In [ ]:
# Perform distortion correction on one camera calibration image
img = mpimg.imread('camera_cal/calibration1.jpg')
# Test undistortion on image
dst = camera_undistort(img, mtx, dist)
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
f.subplots_adjust(hspace = .2, wspace=.05)
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(dst)
ax2.set_title('Correcting the distortion', fontsize=30)


In [ ]:
# Perform distortion correction on test images
images = glob.glob('test_images/test*.jpg')
for image in images:
    img = mpimg.imread(image)    
    # Test undistortion on image
    dst = camera_undistort(img, mtx, dist)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    f.subplots_adjust(hspace = .2, wspace=.05)
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=30)
    ax2.imshow(dst)
    ax2.set_title('correcting the distortion', fontsize=30)

# Perspective Transform


In [ ]:
# Define a function that takes an image, number of x and y points, 
# camera matrix and distortion coefficients
def corners_unwarp(img, nx, ny, mtx, dist):
    # Use the OpenCV undistort() function to remove distortion
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    # Convert undistorted image to grayscale
    gray = cv2.cvtColor(undist, cv2.COLOR_BGR2GRAY)
    # Search for corners in the grayscaled image
    # ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
    # global warped
    # if ret == True:
    # If we found corners, draw them! (just for fun)
    h,w = gray.shape
    # cv2.drawChessboardCorners(undist, (nx, ny), corners, ret)
    # Choose offset from image corners to plot detected corners
    # This should be chosen to present the result at the proper aspect ratio
    # For source points I'm grabbing the outer four detected corners
    src = np.float32([[575,464],[707,464],[258,682],[1049,682]])
    # For destination points, I'm arbitrarily choosing some points to be
    # a nice fit for displaying our warped result 
    # again, not exact, but close enough for our purposes
    dst = np.float32([[450,0],[w-450,0],[450,h],[w-450,h]])
    # Given src and dst points, calculate the perspective transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    # Warp the image using OpenCV warpPerspective()
    return cv2.warpPerspective(undist, M, gray.shape[::-1], flags = cv2.INTER_LINEAR), M, Minv

# Return the resulting image and matrix


In [ ]:
# Perform distortion correction on one camera calibration image
img = mpimg.imread('test_images/test3.jpg')
# Test the corners_unwarp on test image
unwarp_img, M, Minv = corners_unwarp(img, nx, ny, mtx, dist)
# Minv is used at the end for projecting the path onto real world
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
f.subplots_adjust(hspace = .2, wspace=.05)
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(unwarp_img)
ax2.set_title('Unwarped_Image', fontsize=30)


# Applying Sobel

In [ ]:
# Define a function that applies Sobel x or y, 
# then takes an absolute value and applies a threshold.
# Note: calling your function with orient='x', thresh_min=5, thresh_max=100
# should produce output like the example image shown above this quiz.
def abs_sobel_thresh(img, orient='x', thresh_min=25, thresh_max=210):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    # 3) Take the absolute value of the derivative or gradient
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    # 5) Create a mask of 1's where the scaled gradient magnitude 
            # is > thresh_min and < thresh_max
    # 6) Return this mask as your binary_output image
    # binary_output = np.copy(img) # Remove this line
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    # Return the result
    return binary_output
    


In [ ]:
def debug(min_thresh, max_thresh):
    # Run the function
    grad_binary = abs_sobel_thresh(unwarp_img, orient='x', thresh_min=min_thresh, thresh_max=max_thresh)
    # Plot the result
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(unwarp_img)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(grad_binary, cmap='gray')
    ax2.set_title('Thresholded Gradient', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    
interact(debug, min_thresh=(0,255), max_thresh=(0,255))   

In [ ]:
# threshold levels set as (25,210)

In [ ]:
def mag_thresh(img, sobel_kernel=9, mag_thresh_min=30, mag_thresh_max = 100):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    # 2) Take the gradient in x and y separately
    # 3) Calculate the magnitude 
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    # 5) Create a binary mask where mag thresholds are met
    # 6) Return this mask as your binary_output image
     # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh_min) & (gradmag <= mag_thresh_max)] = 1

    # Return the binary image
    return binary_output

In [ ]:
def debug(kernel_size, min_thresh, max_thresh):
    # Run the function
    mag_binary = mag_thresh(unwarp_img, sobel_kernel=kernel_size, mag_thresh_min = min_thresh, mag_thresh_max = max_thresh)
    # Plot the result
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(unwarp_img)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(mag_binary, cmap='gray')
    ax2.set_title('Thresholded Magnitude', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    
interact(debug, kernel_size=(1,11,2), min_thresh=(0,255),max_thresh=(0,255))


In [ ]:
# Kernel size = 9, thresh=(30,100)

In [ ]:
# Define a function that applies Sobel x and y, 
# then computes the direction of the gradient
# and applies a threshold.
def dir_threshold(img, sobel_kernel=15, thresh_min=0.70, thresh_max = 1.30):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    # 2) Take the gradient in x and y separately
    # 3) Take the absolute value of the x and y gradients
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    # 5) Create a binary mask where direction thresholds are met
    # 6) Return this mask as your binary_output image
    # binary_output = np.copy(img) # Remove this line
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh_min) & (absgraddir <= thresh_max)] = 1

    # Return the binary image
    return binary_output
    


In [ ]:
def debug(kernel_size, min_thresh, max_thresh):
    # Run the function
    dir_binary = dir_threshold(unwarp_img, sobel_kernel=kernel_size, thresh_min=min_thresh, thresh_max = max_thresh)
    # Plot the result
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(unwarp_img)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(dir_binary, cmap='gray')
    ax2.set_title('Thresholded Grad. Dir.', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

interact(debug, kernel_size=(1,21,2),min_thresh=(0,np.pi/2,0.01),max_thresh=(0,np.pi/2,0.01))

In [ ]:
# Kernel_size = 15, threshold=(0.7,1.3)

In [ ]:
def hls_channel(img, channel_index = 2, thresh_min = 90,thresh_max = 255):
    # Convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    
    # Extract the desired channel
    channel = hls[:,:,channel_index]
    
    # Apply the threshold
    binary_s = np.zeros_like(channel)
    binary_s[(channel>=thresh_min)&(channel<=thresh_max)]=1
    return binary_s

In [ ]:
def debug(name_channel, min_thresh, max_thresh):
    binary_s = hls_channel(unwarp_img, channel_index = name_channel, thresh_min = min_thresh, thresh_max = max_thresh)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    f.subplots_adjust(hspace = .2, wspace=.05)
    ax1.imshow(unwarp_img,cmap = 'gray')
    ax1.set_title('test Image', fontsize=30)
    ax2.imshow(binary_s,cmap = 'gray')
    ax2.set_title(str(name_channel)+'channel of hls Binary Image', fontsize=30)
    
interact(debug,name_channel=(0,2,1), min_thresh = (0,255),max_thresh = (0,255))

In [ ]:
# currently using only s channel as mentioned by instructor threshold = (90,255).
# I am also using H channel thresh=(15,106)

In [ ]:
# define pipeline
"""
Functions
i. camera_undistort(img, mtx, dist)
ii.corners_unwarp(img, nx, ny, mtx, dist)
iii. abs_sobel_thresh(img, orient='x', thresh_min=25, thresh_max=210
iv.mag_thresh(img, sobel_kernel=9, mag_thresh_min=30, mag_thresh_max = 100)
v. dir_threshold(img, sobel_kernel=15, thresh_min=0.70, thresh_max = 1.30)
vi. hls_channel(img, channel_index = 2, thresh_min = 90,thresh_max = 255)
"""
def pipeline(image):
    img = camera_undistort(image,mtx,dist)
    unwarp_img, M, Minv = corners_unwarp(img,nx,ny,mtx,dist)
    s_channel = hls_channel(unwarp_img)
    h_channel = hls_channel(unwarp_img,channel_index = 0, thresh_min = 25, thresh_max = 90)
    sobelAbs = abs_sobel_thresh(unwarp_img)
    sobelMag = mag_thresh(unwarp_img)
    sobelDir = dir_threshold(unwarp_img)
    combo = np.zeros_like(sobelAbs)
    combo[((s_channel==1)|(h_channel==1))|((sobelAbs==1)|(sobelMag==1))]=1
          #|(sobelDir==1)]=1
    return combo, Minv
    

In [ ]:
# Perform the combination of all features on test images
images = glob.glob('test_images/test*.jpg')
for image in images:
    img = mpimg.imread(image)    
    # Test undistortion on image
    combination, Minv = pipeline(img)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    f.subplots_adjust(hspace = .2, wspace=.05)
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=30)
    ax2.imshow(combination,cmap='gray')
    ax2.set_title('correction', fontsize=30)

In [ ]:
image = mpimg.imread('test_images/test3.jpg')
combination, Minv = pipeline(image)
plt.figure()
plt.imshow(image)
plt.figure()
plt.imshow(combination,cmap='gray')
plt.figure()
histogram = np.sum(combination[combination.shape[0]/2:,:], axis=0)
plt.plot(histogram)

In [ ]:
binary_warped = combination
# Create an output image to draw on and  visualize the result
out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
# Find the peak of the left and right halves of the histogram
# These will be the starting point for the left and right lines
midpoint = np.int(histogram.shape[0]/2)
leftx_base = np.argmax(histogram[:midpoint])
rightx_base = np.argmax(histogram[midpoint:]) + midpoint

# Choose the number of sliding windows
nwindows = 9
# Set height of windows
window_height = np.int(binary_warped.shape[0]/nwindows)
# Identify the x and y positions of all nonzero pixels in the image
nonzero = binary_warped.nonzero()
nonzeroy = np.array(nonzero[0])
nonzerox = np.array(nonzero[1])
# Current positions to be updated for each window
leftx_current = leftx_base
rightx_current = rightx_base
# Set the width of the windows +/- margin
margin = 100
# Set minimum number of pixels found to recenter window
minpix = 50
# Create empty lists to receive left and right lane pixel indices
left_lane_inds = []
right_lane_inds = []

# Step through the windows one by one
for window in range(nwindows):
    # Identify window boundaries in x and y (and right and left)
    win_y_low = binary_warped.shape[0] - (window+1)*window_height
    win_y_high = binary_warped.shape[0] - window*window_height
    win_xleft_low = leftx_current - margin
    win_xleft_high = leftx_current + margin
    win_xright_low = rightx_current - margin
    win_xright_high = rightx_current + margin
    # Draw the windows on the visualization image
    cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
    cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
    # Identify the nonzero pixels in x and y within the window
    good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
    good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
    # Append these indices to the lists
    left_lane_inds.append(good_left_inds)
    right_lane_inds.append(good_right_inds)
    # If you found > minpix pixels, recenter next window on their mean position
    if len(good_left_inds) > minpix:
        leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
    if len(good_right_inds) > minpix:        
        rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

# Concatenate the arrays of indices
left_lane_inds = np.concatenate(left_lane_inds)
right_lane_inds = np.concatenate(right_lane_inds)

# Extract left and right line pixel positions
leftx = nonzerox[left_lane_inds]
lefty = nonzeroy[left_lane_inds] 
rightx = nonzerox[right_lane_inds]
righty = nonzeroy[right_lane_inds] 

# Fit a second order polynomial to each
left_fit = np.polyfit(lefty, leftx, 2)
right_fit = np.polyfit(righty, rightx, 2)

In [ ]:
# Generate x and y values for plotting
ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
plt.imshow(out_img)
plt.plot(left_fitx, ploty, color='yellow')
plt.plot(right_fitx, ploty, color='yellow')
plt.xlim(0, 1280)
plt.ylim(720, 0)

In [ ]:
# Measuring curvature
# Generate some fake data to represent lane-line pixels
ploty = np.linspace(0, 719, num=720)# to cover same y-range as image
quadratic_coeff = 3e-4 # arbitrary quadratic coefficient
# For each y position generate random x position within +/-50 pix
# of the line base position in each case (x=200 for left, and x=900 for right)
leftx = np.array([200 + (y**2)*quadratic_coeff + np.random.randint(-50, high=51) 
                              for y in ploty])
rightx = np.array([900 + (y**2)*quadratic_coeff + np.random.randint(-50, high=51) 
                                for y in ploty])

leftx = leftx[::-1]  # Reverse to match top-to-bottom in y
rightx = rightx[::-1]  # Reverse to match top-to-bottom in y


# Fit a second order polynomial to pixel positions in each fake lane line
left_fit = np.polyfit(ploty, leftx, 2)
left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fit = np.polyfit(ploty, rightx, 2)
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

# Plot up the fake data
mark_size = 3
plt.plot(leftx, ploty, 'o', color='red', markersize=mark_size)
plt.plot(rightx, ploty, 'o', color='blue', markersize=mark_size)
plt.xlim(0, 1280)
plt.ylim(0, 720)
plt.plot(left_fitx, ploty, color='green', linewidth=3)
plt.plot(right_fitx, ploty, color='green', linewidth=3)
plt.gca().invert_yaxis() # to visualize as we do the images

In [ ]:
# Define y-value where we want radius of curvature
# I'll choose the maximum y-value, corresponding to the bottom of the image
y_eval = np.max(ploty)
left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
print(left_curverad, right_curverad)
# Example values: 1926.74 1908.48

In [ ]:
# Define conversions in x and y from pixels space to meters
ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/700 # meters per pixel in x dimension

# Fit new polynomials to x,y in world space
left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)
# Calculate the new radii of curvature
left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
# Now our radius of curvature is in meters
print(left_curverad, 'm', right_curverad, 'm')
# Example values: 632.1 m    626.2 m